In [5]:
from __future__ import print_function
import pandas as pd
import numpy as np
import datetime as dt
import quandl

# Data 

In [6]:
assets=pd.read_csv('SP500.txt', comment='#').set_index('Symbol')

QUANDL={
    ## Get a key (free) from quandl.com and copy it here
    'authtoken':"",
    'start_date':dt.date(2007, 1, 1),
    'end_date':dt.date(2016, 12, 31)
}
RISK_FREE_SYMBOL = "USDOLLAR"
data={}

#### Download loop

If it stops because of Quandl error codes 503 or 504, try re-running it (it won't download data already downloaded).

In [40]:
# download assets' data
for ticker in assets.index:
    if ticker in data:
        continue
    print('downloading %s from %s to %s' %(ticker, QUANDL['start_date'], QUANDL['end_date']))
    try:
        data[ticker] = quandl.get(assets.Quandlcode[ticker], **QUANDL)
    except quandl.NotFoundError:
        print('\tInvalid asset code')

downloading BRK-B from 2007-01-01 to 2016-12-31
	Invalid asset code
downloading BF-B from 2007-01-01 to 2016-12-31
	Invalid asset code


#### Computation 

In [30]:
def select_first_valid_column(df, columns):
    for column in columns:
        if column in df.columns:
            return df[column]
        
# extract prices
prices=pd.DataFrame.from_items([(k,select_first_valid_column(v, ["Adj. Close", "Close", "VALUE"])) 
                                for k,v in data.items()])

#compute sigmas
high=pd.DataFrame.from_items([(k,select_first_valid_column(v, ["High"])) for k,v in data.items()])
low=pd.DataFrame.from_items([(k,select_first_valid_column(v, ["Low"])) for k,v in data.items()])
sigmas = (high-low) / (2*high)

# extract volumes
volumes=pd.DataFrame.from_items([(k,select_first_valid_column(v, ["Adj. Volume", "Volume"])) for k,v in data.items()])

# fix risk free
prices[RISK_FREE_SYMBOL]=10000*(1 + prices[RISK_FREE_SYMBOL]/(100*250)).cumprod()

#### Filtering 

In [31]:
# filter NaNs - threshold at 2% missing values
bad_assets = prices.columns[prices.isnull().sum()>len(prices)*0.02]
if len(bad_assets):
    print('Assets %s have too many NaNs, removing them' % bad_assets)

prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]

nassets=prices.shape[1]

# days on which many assets have missing values
bad_days1=sigmas.index[sigmas.isnull().sum(1) > nassets*.9]
bad_days2=prices.index[prices.isnull().sum(1) > nassets*.9]
bad_days3=volumes.index[volumes.isnull().sum(1) > nassets*.9]
bad_days=pd.Index(set(bad_days1).union(set(bad_days2)).union(set(bad_days3))).sort_values()
print ("Removing these days from dataset:")
print(pd.DataFrame({'nan price':prices.isnull().sum(1)[bad_days],
                    'nan volumes':volumes.isnull().sum(1)[bad_days],
                    'nan sigmas':sigmas.isnull().sum(1)[bad_days]}))

prices=prices.loc[~prices.index.isin(bad_days)]
sigmas=sigmas.loc[~sigmas.index.isin(bad_days)]
volumes=volumes.loc[~volumes.index.isin(bad_days)]

# extra filtering
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))
prices=prices.fillna(method='ffill')
sigmas=sigmas.fillna(method='ffill')
volumes=volumes.fillna(method='ffill')
print(pd.DataFrame({'remaining nan price':prices.isnull().sum(),
                    'remaining nan volumes':volumes.isnull().sum(),
                    'remaining nan sigmas':sigmas.isnull().sum()}))

Assets Index(['ABBV', 'ACE', 'ADT', 'GAS', 'ARG', 'AA', 'ALLE', 'GOOG', 'ALTR',
       'AVGO', 'BXLT', 'BRCM', 'CVC', 'CAM', 'CCE', 'CPGX', 'CMCSK', 'CSC',
       'CNX', 'DLPH', 'DAL', 'DFS', 'DISCK', 'DG', 'DPS', 'EMC', 'ESV', 'FB',
       'FOSL', 'GM', 'GNW', 'HCA', 'HCBK', 'GMCR', 'KMI', 'KHC', 'LYB', 'MNK',
       'MPC', 'MHFI', 'MJN', 'WRK', 'KORS', 'NAVI', 'NWSA', 'NWS', 'NLSN',
       'PYPL', 'POM', 'PM', 'PSX', 'PCL', 'PCP', 'QRVO', 'SNDK', 'SNI', 'SIAL',
       'HOT', 'TEL', 'TE', 'TGNA', 'THC', 'TDC', 'TWC', 'TRIP', 'TYC', 'VRSK',
       'V', 'WBA', 'ANTM', 'XYL', 'ZBH', 'ZTS'],
      dtype='object') have too many NaNs, removing them
Removing these days from dataset:
            nan price  nan sigmas  nan volumes
2007-01-02        430         NaN          NaN
2007-04-06        430         NaN          NaN
2007-05-27        431       431.0        430.0
2010-04-02        430         NaN          NaN
2012-04-06        430         NaN          NaN
2012-10-29        430       431.

#### Save 

In [33]:
# make volumes in dollars
volumes = volumes*prices

# compute returns
returns = (prices.diff()/prices.shift(1)).fillna(method='ffill').ix[1:]

bad_assets = returns.columns[((-.5>returns).sum()>0)|((returns > 2.).sum()>0)]
if len(bad_assets):
    print('Assets %s have dubious returns, removed' % bad_assets)
    
prices = prices.loc[:,~prices.columns.isin(bad_assets)]
sigmas = sigmas.loc[:,~sigmas.columns.isin(bad_assets)]
volumes = volumes.loc[:,~volumes.columns.isin(bad_assets)]
returns = returns.loc[:,~returns.columns.isin(bad_assets)]

# save data
prices.to_csv('prices.csv.gz', compression='gzip', float_format='%.3f')
volumes.to_csv('volumes.csv.gz', compression='gzip', float_format='%d')
returns.to_csv('returns.csv.gz', compression='gzip', float_format='%.3e')
sigmas.to_csv('sigmas.csv.gz', compression='gzip', float_format='%.3e')

Assets Index(['AAL', 'AIG', 'DISCA', 'ETFC', 'EBAY', 'GGP', 'HBI', 'HIG', 'PPG',
       'STT', 'XL'],
      dtype='object') have dubious returns


### Data we generate 

In [4]:
a = pd.DataFrame(data=2.5*1e-4, index=volumes.index, columns=volumes.columns)
b = pd.DataFrame(data=1, index=volumes.index, columns=volumes.columns) 
s = pd.DataFrame(data=1*1e-4, index=volumes.index, columns=volumes.columns)

a.to_csv('a.csv.gz', compression='gzip', float_format='%g')
b.to_csv('b.csv.gz', compression='gzip', float_format='%g')
s.to_csv('s.csv.gz', compression='gzip', float_format='%g')

# Estimates 

In [9]:
np.random.seed(0)
noise=pd.DataFrame(index=returns.index, columns=returns.columns, 
                   data=0.1*np.random.randn(*returns.values.shape))
return_estimate= returns + noise
return_estimate.USDOLLAR = returns.USDOLLAR

return_estimate.to_csv('return_estimate.csv.gz', compression='gzip', float_format='%.3e')

volumes.rolling(window=10, center=False).mean().to_csv('volume_estimate.csv.gz', 
                                                       compression='gzip', float_format='%d')
sigmas.rolling(window=10, center=False).mean().to_csv('sigma_estimate.csv.gz', 
                                                       compression='gzip', float_format='%.3e')

# Risk model 

In [40]:
start_t="2012-01-01"
end_t="2016-12-31"

first_days_month=\
    pd.date_range(start=returns.index[next(i for (i,el) in enumerate(returns.index >= start_t) if el)-1],
                                 end=returns.index[-1], freq='MS')

k=15
exposures, factor_sigma, idyos = {}, {},{}
for day in first_days_month:
    used_returns = returns.loc[(returns.index < day)&
                        (returns.index >= day-pd.Timedelta("730 days"))]
    second_moment=used_returns.values.T@used_returns.values/used_returns.values.shape[0]
    eival,eivec=np.linalg.eigh(second_moment)
    factor_sigma[day]=np.diag(eival[-k:])
    exposures[day]=pd.DataFrame(data=eivec[:,-k:],index=returns.columns)
    idyos[day]=pd.Series(data=np.diag(eivec[:,:-k]@np.diag(eival[:-k])@eivec[:,:-k].T),index=returns.columns)
    
pd.Panel(exposures).swapaxes(1,2).to_hdf('risk_model.h5', 'exposures')
pd.DataFrame(idyos).T.to_hdf('risk_model.h5', 'idyos')
pd.Panel(factor_sigma).to_hdf('risk_model.h5', 'factor_sigma')

# load with
# store = pd.HDFStore('risk_model.h5')
# store.exposures, etc.